In [48]:
"""
Q1: For which of these matrices (A1, A2) does the jacobi iteration and Gauss-Siedel method converge?
"""
import numpy as np
from scipy.linalg import inv, lu
from math import sqrt, cos, pi
import scipy.sparse.linalg as sp
from scipy import sparse

In [49]:
A1 = [[2,-1,2],[1,2,-2],[2,2,2]]
A2 = [[5,5,0],[-1,5,4],[2,3,8]]

In [50]:
"""
Convergence Tests: Every eigenvalue of M=I-P^-1A must have |eigen(M)| < 1
Jacobi Iteration: P = diagonal part D of A
Gauss-Siedel: P = Lower Triangular part of A
"""
def jacobiConvergence(A):
    m, n = len(A), len(A[0])
    
    #constuct Diagonal and Identity matrix
    D = [[0 for x in range(n)] for y in range(m)]
    I = [[0 for x in range(n)] for y in range(m)]
    for x in range(m):
        D[x][x] = A[x][x]
        I[x][x] = 1
        
    #turn into numpy arrays
    A = np.array(A)
    D = np.array(D)
    I = np.array(I)
    
    M = I - (inv(D).dot(A))
    
    w, v = np.linalg.eig(M)
    
    for eigen_val in w: 
        if (abs(eigen_val) >= 1):
            print("Does not converge")
            return
    print("Converges")

def guasssiedelConvergence(A):
    m, n = len(A), len(A[0])
    
    #construct L and Identity matrix
    P, L, U = lu(A)
    I = [[0 for x in range(n)] for y in range(m)]
    for x in range(m):
        I[x][x] = 1
        
    M = I - (inv(L).dot(A))
    
    w, v = np.linalg.eig(M)
    
    for eigen_val in w: 
        if (abs(eigen_val) >= 1):
            print("Does not converge")
            return
    print("Converges")

In [51]:
print("Testing Matrix A1")
print("Jacobi Iteration")
jacobiConvergence(A1)
print("Gauss-Siedel")
guasssiedelConvergence(A1)
print("\n")
print("Testing Matrix A2")
print("Jacobi Iteration")
jacobiConvergence(A2)
print("Gauss-Siedel")
guasssiedelConvergence(A2)

Testing Matrix A1
Jacobi Iteration
Does not converge
Gauss-Siedel
Does not converge


Testing Matrix A2
Jacobi Iteration
Converges
Gauss-Siedel
Does not converge


In [53]:
"""
QII: Consider two different fixed-pt iterations to compute the inverse of a regular matrix A(nxn)
     starting from an initial value X(0)
     (1) X^k = X^k-1(I-AC)+C, k=1,2....
     (2) X^k = X^k-1(2I-AX^k-1)
"""
A = np.array([[1,2,3],[4,5,6],[7,8,10]])
C = np.array([[-0.5,-1.5,1],[-0.5,3.7,-2],[1,-2,1]])
I = [[0 for x in range(3)] for y in range(3)]
for x in range(3):
    I[x][x] = 1
I = np.array(I)

In [54]:
"""
(a) What are sufficient conditions for convergence for these iterations?
    What convergence order do you expect?
    
converges for any initial value X0 iff max|eigen value| < 1
r = spr(I-C^-1A)
convergence order is linear ||x^k+1-x|| <= r ||x^k - x||
"""
def testConvergence():
    matrix = I-(inv(C).dot(A))
    w, v = np.linalg.eig(matrix)
    for eigen_val in w: 
        if (abs(eigen_val) >= 1):
            print("Does not converge for any initial value x0")
            return
    print("Converges") 

testConvergence()

Does not converge for any initial value x0


In [55]:
"""
(b) X^0 = I and X^0 = C
"""
def fixedPtIterations(A, C):    
    print("Inverse of A")
    print(inv(A))
    print("\n")


    def iterate(X0, start, end):
        print("Fixed-point iteration: X^k = X^k-1(I-AC)+C")
        X = X0
        k = start
        while (k <= end):
            X = X.dot((I-A.dot(C))) + C
            k += 1
        print(X)
        
        print("Fixed-point iteration: X^k = X^k-1(2I-AX^k-1)")
        X = X0
        k = start
        while (k <= end):
            X = X.dot((2*I)-(A.dot(X)))
            k += 1
        print(X)
    
    #testing with initial value X0 = I
    print("Initial value X0 = I")
    iterate(I, 1, 100)
    print("\n")
    #testing with initial value X0 = C
    print("Initial value X0 = C")
    iterate(C, 1, 100)

In [56]:
fixedPtIterations(A, C)

Inverse of A
[[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]


Initial value X0 = I
Fixed-point iteration: X^k = X^k-1(I-AC)+C
[[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]
Fixed-point iteration: X^k = X^k-1(2I-AX^k-1)
[[-2102729259 -1313886708  -601390378]
 [ 1148497904  2043996001  -165388804]
 [-1057446898  -393417064  -697791545]]


Initial value X0 = C
Fixed-point iteration: X^k = X^k-1(I-AC)+C
[[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]
Fixed-point iteration: X^k = X^k-1(2I-AX^k-1)
[[-0.66666667 -1.33333333  1.        ]
 [-0.66666667  3.66666667 -2.        ]
 [ 1.         -2.          1.        ]]


In [4]:
"""
QIII: Consider the m-dependent matrix A. Solve the system Ax=b with
      (a) Jacobi Method
      (b) Gauss-Siedel Method
      (c) SOR method
      Use m = 2^k, k=1,..,6
      Stopping criterion use ||b-Ax^k||inf <= 10^-8||x^k||inf or k >= 20000
      Use given relaxation parameter for the SOR method
"""
def createMatrixA(m):
    n = m * m 
    A = [[0 for x in range(n)] for y in range(n)]
    i = 0
    while i < n:
        if i + m < n:
            for j in range(1, m):
                A[i+m][i] = -1
                A[i+m+j][i+j] = -1
                A[i][i+m] = -1
                A[i+j][i+m+j] = -1
        for j in range(m):
            A[i+j][i+j] = 4
        for j in range(m-1):
            A[i+j+1][i+j] = -1
            A[i+j][i+j+1] = -1
        i += m
    return A

def createMatrixb(m):
    n = m * m
    b = [[1] for x in range(n)]
    return b

In [5]:
def jacobiMethod(A, b, x=None):
    if x is None:
        x = np.zeros_like(b)
    
    D = np.diag(A)
    R = A - np.diagflat(D)
    
    k = 1
    while ((np.linalg.norm(b-(A.dot(x)), np.inf) > (1e-8)*np.linalg.norm(x, np.inf)) and k < 20000):
        x = (b-np.dot(R,x)) / D
        k += 1
    return k

def gaussSiedel(A, b, x=None):
    [m, n] = np.shape(A)
    
    P, L, U = lu(A)
    x = np.ones((m,1))
    
    k = 1
    while ((np.linalg.norm(b-(A.dot(x)), np.inf) > (1e-8)*np.linalg.norm(x, np.inf)) and k < 20000):
        x_new = np.dot(inv(L), (b-np.dot(U, x)))
        x = x_new
        k += 1
    return k

def sorMethod(A, b, x, omega):
    phi = x[:]
    k = 1
    while ((np.linalg.norm(b-(A.dot(phi)), np.inf) > (1e-8)*np.linalg.norm(phi, np.inf)) and k < 20000):
        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i][j] * phi[j]
            phi[i] = (1 - omega) * phi[i] + (omega / A[i][i]) * (b[i] - sigma)
        k += 1
    return k
    

In [10]:
m_vals = [2**k for k in range(1,6)]
    
print("Iterations required as a function of m")
for m in m_vals:
    print("m: " + str(m))
    A = np.array(createMatrixA(m))
    b = np.array(createMatrixb(m))
    print("Jacobi Method: {}".format(jacobiMethod(A, b)))
    print("Gauss-Siedel: {}".format(gaussSiedel(A, b)))
    h = 1 / (m + 1)
    sprJ = cos(pi*h)
    omega = 2 / (1 + sqrt(1-(sprJ**2)))
    print("SOR Method: {}".format(sorMethod(A, b, np.zeros(m*m), omega)))
    print("\n")
print("*Note: Could no get m=64 case to run on computer")

    

Iterations required as a function of m
m: 2
Jacobi Method: 29
Gauss-Siedel: 540
SOR Method: 11


m: 4
Jacobi Method: 87
Gauss-Siedel: 551
SOR Method: 20


m: 8
Jacobi Method: 276
Gauss-Siedel: 572
SOR Method: 37


m: 16
Jacobi Method: 924
Gauss-Siedel: 586
SOR Method: 70


m: 32
Jacobi Method: 3200
Gauss-Siedel: 596
SOR Method: 135


*Note: Could no get m=64 case to run on computer


In [41]:
"""
QIV: Same criteria as QIII
     (d) Gradient Method
     (e) Conjugate Gradient Method
     (f) Preconditioned Conjugate Gradient Method
"""
def gradientMethod(A, b, x):
    def f(A, b, x):
        return 0.5*np.dot(np.dot(x,A),x)-np.dot(x,b)
    def df(A, b, x):
        return np.dot(A,x)-b
    
    new_x = 6 * x #start initial guess vector at 6
    k, MAX_ITERATIONS, gamma = 1, 20000, 0.01
    stop = False
    while (k < MAX_ITERATIONS and not stop):
        curr_x = new_x
        new_x = curr_x - gamma * df(A,b,curr_x)
        
        step = new_x - curr_x
        
        if np.linalg.norm(b - A.dot(new_x), np.inf) <= 1e-8*np.linalg.norm(new_x, np.inf):
            stop = True
            break
        
        k += 1
    return k

def conjGradMethod(A, b, x):
    r = b - np.dot(A, x)
    p = r
    rs = np.dot(np.transpose(r), r)
    k, MAX_ITERATIONS = 1, 20000
    stop = False
    while (k < MAX_ITERATIONS and not stop):
        Ap = np.dot(A, p)
        alpha = rs / np.dot(np.transpose(p), Ap)
        x += alpha * p
        r -= alpha * Ap
        rs_new = np.dot(np.transpose(r), r)
        if np.linalg.norm(b - A.dot(x), np.inf) <= 1e-8*np.linalg.norm(x, np.inf):
            stop = True
            break
        p = r + (rs_new/rs)*p
        rs = rs_new
        k += 1
    return k

def preCondConjGradMethod(A, b, x, n):
    M_inv = sp.spilu(sparse.csr_matrix(A))
    M2 = sp.LinearOperator((n,n), M_inv.solve)
    x3 = sp.cg(A,b,M=M2)
    num_iters = 0
    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    sp.cg(A, b, tol=1e-8, M=M2, callback=callback)
    return num_iters
    

In [44]:
print("Iterations required as a function of m")
for m in m_vals:
    print("m: " + str(m))
    A = np.array(createMatrixA(m))
    b = np.array(createMatrixb(m))
    x = np.ones((m*m,1))
    print("Gradient Method: {}".format(gradientMethod(A, b, x)))
    print("Conjugate Gradient Method: {}".format(conjGradMethod(A, b, x)))
    print("Pre-Cond Conjugate Gradient Method: {}".format(preCondConjGradMethod(A, b, x, m*m)))
    print("\n")
print("*Note: Could no get m=64 case to run on computer")

Iterations required as a function of m
m: 2
Gradient Method: 1065
Conjugate Gradient Method: 1
Pre-Cond Conjugate Gradient Method: 1


m: 4
Gradient Method: 2544
Conjugate Gradient Method: 15
Pre-Cond Conjugate Gradient Method: 1


m: 8
Gradient Method: 6746
Conjugate Gradient Method: 108
Pre-Cond Conjugate Gradient Method: 2


m: 16
Gradient Method: 20000
Conjugate Gradient Method: 419
Pre-Cond Conjugate Gradient Method: 3


m: 32
Gradient Method: 20000
Conjugate Gradient Method: 1701
Pre-Cond Conjugate Gradient Method: 4


*Note: Could no get m=64 case to run on computer
